In [83]:
## 请安装前置依赖
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import json
## 安装完别忘了添加以下下面 todo 的参数
## 比较隐私的内容请做好保护


In [84]:
def match_the_type(type_name):
    if( type_name == "Gene"):
        return "Genetic"
    else:
        return type_name

In [85]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    # 创建节点
    def create_node(self, node_data):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = ""
            if( 'alias' in node_data['content']):
                result = session.execute_write(
                    self._create_node_with_alias, node_data['content']['type'], node_data['name'], node_data['content']['alias']
                )
            else:
                result = session.execute_write(
                    self._create_node_without_alias, node_data['content']['type'], node_data['name']
                )
            
            assert(result != "")
            for row in result:
                print("Created node: {node_name}".format(node_name=row['name']))

    @staticmethod
    def _create_node_with_alias(tx, node_type,node_name,node_alias):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = ()
        match node_type:
            case 'disease':
                query = (
                    "CREATE (p1: Disease { name: $node_name, alias: $node_alias }) "
                    "RETURN p1"
                )
            case 'chemical':
                query = (
                    "CREATE (p1: Chemical { name: $node_name, alias: $node_alias }) "
                    "RETURN p1"
                )
            case 'genetic':
                query = (
                    "CREATE (p1: Genetic { name: $node_name, alias: $node_alias }) "
                    "RETURN p1"
                )
            case 'treatment':
                query = (
                    "CREATE (p1: Treatment { name: $node_name, alias: $node_alias }) "
                    "RETURN p1"
                )
        assert(query != ())
        result = tx.run(query,node_name=node_name,node_alias=node_alias)
        try:
            return [{"name": row["p1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
    
    @staticmethod
    def _create_node_without_alias(tx, node_type,node_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = ()
        match node_type.lower():
            case 'disease':
                query = (
                    "CREATE (p1: Disease { name: $node_name }) "
                    "RETURN p1"
                )
            case 'chemical':
                query = (
                    "CREATE (p1: Chemical { name: $node_name }) "
                    "RETURN p1"
                )
            case 'genetic':
                query = (
                    "CREATE (p1: Genetic { name: $node_name }) "
                    "RETURN p1"
                )
            case 'gene':
                query = (
                    "CREATE (p1: Genetic { name: $node_name }) "
                    "RETURN p1"
                )
            case 'variant':
                query = (
                    "CREATE (p1: Variant { name: $node_name }) "
                    "RETURN p1"
                )
            case 'treatment':
                query = (
                    "CREATE (p1: Treatment { name: $node_name }) "
                    "RETURN p1"
                )
        assert(query != ())
        result = tx.run(query,node_name=node_name)
        try:
            return [{"name": row["p1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
    
    def find_and_create_node(self, node_data):
        with self.driver.session(database="neo4j") as session:
            result = []
            match(node_data['content']['type']):
                case 'Disease':
                    result = session.execute_read(self._find_and_return_disease_node, node_data['name'])
                case 'Gene':
                    result = session.execute_read(self._find_and_return_genetic_node, node_data['name'])
                case 'Chemical':
                    result = session.execute_read(self._find_and_return_chemical_node, node_data['name'])
                case 'Variant':
                    result = session.execute_read(self._find_and_return_variant_node, node_data['name']) 
            if(result != []):
                for row in result:
                    print(row)
            else:
                self.create_node(node_data=node_data)
                return 
    
    @staticmethod
    def _find_and_return_disease_node(tx, node_name):
        # 执行语句
        query = (
            "MATCH (p: Disease) "
            "WHERE p.name = $node_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, node_name=node_name)
        return [row["name"] for row in result]
    @staticmethod
    def _find_and_return_chemical_node(tx, node_name):
        # 执行语句
        query = (
            "MATCH (p: Chemical) "
            "WHERE p.name = $node_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, node_name=node_name)
        return [row["name"] for row in result]
    @staticmethod
    def _find_and_return_genetic_node(tx, node_name):
        # 执行语句
        query = (
            "MATCH (p: Genetic) "
            "WHERE p.name = $node_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, node_name=node_name)
        return [row["name"] for row in result]
    @staticmethod
    def _find_and_return_variant_node(tx, node_name):
        # 执行语句
        query = (
            "MATCH (p: Variant) "
            "WHERE p.name = $node_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, node_name=node_name)
        return [row["name"] for row in result]

    def create_relationship(self,node1_data,node2_data,relationship_data):
        with self.driver.session(database="neo4j") as session:
            summary = session.execute_write(self._create_relationship_without_attribute,node1_data,node2_data,relationship_data['name'])
            print(f"Query counters: {summary.counters}.")
    
    @staticmethod
    def _create_relationship_without_attribute(tx,node1_data,node2_data,relationship_name):
        node1_type = match_the_type(node1_data['type'])
        node2_type = match_the_type(node2_data['type'])
        query_str = "MATCH (node1:{node1_type} {{ name:$node1_name }}) MERGE (node2:{node2_type}{{ name:$node2_name }}) MERGE (node1)-[:{relationship}]->(node2)".format(node1_type=node1_type,node2_type=node2_type,relationship=relationship_name)
        query = (query_str)
        result = tx.run(query,
                        node1_name=node1_data['name'],
                        node2_name=node2_data['name'])
        return result.consume()


In [86]:
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
#todo 请添加
uri = "..." # the uri of your neo4j instance
user = "..." # the username you set 
password = "..." # the password you set

app = App(uri, user, password)


In [76]:

## relationship add example json format

# subject_json = {
#     "name": "HBx",
#     "type": "Gene",
# }
# object_json = {
#     "name":"hepatocellular carcinoma",
#     "type":"Disease",
# }
# relationship_json={
#     "name":"Association"
# }

In [20]:
# OK
test_json = {
    'name':"node1",
    "content":{
        'type':'disease',
        'alias':['Node1','node']
    }
}
test_entity = json.loads(json.dumps(test_json))
print(test_entity['content']['type'])

disease


In [ ]:
with open("./ner_output/entities.json","r") as f:
    data = json.load(f)

entity = {
    'name':"",
    'content':{}
}

for key, value in data.items():
    entity['name'] = key
    entity['content'] = value
    app.create_node(entity)

app.close()


In [77]:
with open('./re_output/CMeIE_test.json') as re_f:
    re_data = json.load(re_f)

relation_buf = []

for item in re_data:
    for key,value in item.items():
        if(value):
            for item in value:
                buf = []
                ## buf content explain
                ## buf[0]:subject
                ## buf[1]:subject_type
                ## buf[2]:relation
                ## buf[3]:object
                ## buf[4]:object_type
                buf.append(item['subject'])
                buf.append(item['subject_type'])
                buf.append(item['predicate'])
                buf.append(item['object']['@value'])
                buf.append(item['object_type']['@value'])

                relation_buf.append(buf)

In [87]:
for relations in relation_buf:
    subject_json = {
        "name": relations[0],
        "type": relations[1],
    }
    object_json = {
        "name":relations[3],
        "type":relations[4],
    }
    for relation in relations[2]:
        relationship_json={
            "name":relation
        }
        app.create_relationship(node1_data=subject_json,node2_data=object_json,relationship_data=relationship_json)

app.close()

Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'relationships_created': 1}.
Query counters: {'re

In [ ]:
subject_check_json = {
    'name' : "",
    'content' :{}
}
object_check_json = {
    'name' : "",
    'content' :{}
}
for relation in relation_buf:
    subject_check_json['name'] = relation[0]
    subject_check_json['content']['type'] = relation[1]
    object_check_json['name'] = relation[3]
    object_check_json['content']['type'] = relation[4]
            
    app.find_and_create_node(subject_check_json)
    app.find_and_create_node(object_check_json)

app.close()
    

In [ ]:
relation_json = {}
for relation in relation_buf:
    relation_json['subject'] = relation[0]
    relation_json['subject_type'] = relation[1]
    relation_json['object'] = relation[3]
    relation_json['object_type'] = relation[4]
    relation_json['predicate'] = relation[2]

    print(relation_json)
    break

In [ ]:

test_buf = []
for relation in relation_buf:
    # print(relation[1])
    if( relation[1] not in test_buf ):
        test_buf.append(relation[1])

print(len(test_buf))
print(test_buf)

# 五种关系
# ['Association'(因果), 'Negative'(抑制), 'Correlation'(相关), 'Positive'(促进), 'Comparison'(比较)]
# 四种实体
# ['Gene', 'Chemical', 'Variant', 'Disease']

In [13]:
app.create_node(test_entity)
app.close()

Created node: node1
